# PyCity Schools Analysis

* As a whole, schools with higher budgets, did not yield better test results. By contrast, schools with higher spending per student actually (\$645-675) underperformed compared to schools with smaller budgets (<\$585 per student).

* As a whole, smaller and medium sized schools dramatically out-performed large sized schools on passing math performances (89-91% passing vs 67%).

* As a whole, charter schools out-performed the public district schools across all metrics. However, more analysis will be required to glean if the effect is due to school practices or the fact that charter schools tend to serve smaller student populations per school. 
---

In [78]:
# Dependencies and Setup
import pandas as pd
import numpy as np
import locale as lc

# Get the list of all locale options
all_locales = lc.locale_alias
# Use US Convention
lc.setlocale(lc.LC_ALL,all_locales["en_us"])

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas Data Frames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the overall passing rate (overall average score), i.e. (avg. math score + avg. reading score)/2

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [123]:
# Total Number of Schools
total_schools = school_data['school_name'].count()

# Total Number of Students
total_students = student_data['Student ID'].count()

# Total Budget
total_budget = school_data['budget'].sum()

# Average Math Score
avg_math_score = student_data['math_score'].mean()

# Average Reading Score
avg_read_score = student_data['reading_score'].mean()

# Overall Average Score 
overall_passing_rate = (avg_math_score + avg_read_score) / 2

# Passing Math Score (%)
percent_passing_math = (student_data[student_data["math_score"] >= 70]).count() / total_students
percent_passing_math = percent_passing_math.mean() * 100

# Passing Reading Score (%)
percent_passing_read = (student_data[student_data["reading_score"] >= 70]).count() / total_students
percent_passing_read = percent_passing_read.mean() * 100

# District Summary
district_summary_list = [(total_schools, total_students, total_budget, avg_math_score, avg_read_score, 
                         percent_passing_math, percent_passing_read, overall_passing_rate)]

district_summary_df = pd.DataFrame(district_summary_list, columns=['Total Schools', 'Total Students', 'Total Budget',
                                                                   'Average Math Score', 'Average Reading Score', 
                                                                   '% Passing Math', '% Passing Reading', 
                                                                   '% Overall Passing Rate'])

# Formatting
district_summary_df['Total Budget'] = district_summary_df['Total Budget'].apply(lambda x: "$"+lc.format_string("%.2f",x,True))
district_summary_df['Average Math Score'] = district_summary_df['Average Math Score'].apply(lambda x: lc.format_string("%.6f",x,True))
district_summary_df['Average Reading Score'] = district_summary_df['Average Reading Score'].apply(lambda x: lc.format_string("%.5f",x,True))
district_summary_df['% Passing Math'] = district_summary_df['% Passing Math'].apply(lambda x: lc.format_string("%.6f",x,True))
district_summary_df['% Passing Reading'] = district_summary_df['% Passing Reading'].apply(lambda x: lc.format_string("%.6f",x,True))
district_summary_df['% Overall Passing Rate'] = district_summary_df['% Overall Passing Rate'].apply(lambda x: lc.format_string("%.6f",x,True))

district_summary_df


,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
0,15,39170,"$24,649,428.00",78.985371,81.87784,74.980853,85.805463,80.431606


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)
  
* Create a dataframe to hold the above results

In [126]:
# Group the complete school data by school name and set it as index
school = school_data_complete.set_index('school_name').groupby('school_name')

# School Type
school_type = school_data.set_index('school_name')['type']

# Total Number of Students
total_students = school_data_complete.groupby('school_name')['Student ID'].count()

# Total School Budget
total_school_budget = school_data.set_index('school_name')['budget']

# Per Student Budget
per_student_budget = school_data.set_index('school_name')['budget'] / school_data.set_index('school_name')['size']

# Average Math Score
avg_math_score = school['math_score'].mean()

# Average Reading Score
avg_read_score = school['reading_score'].mean()

# Passing Math Score (%)
percent_passing_math = school_data_complete[school_data_complete['math_score'] >= 70].groupby('school_name')['Student ID'].count() / total_students
percent_passing_math = percent_passing_math * 100

# Passing Reading Score (%)
percent_passing_read = school_data_complete[school_data_complete['reading_score'] >= 70].groupby('school_name')['Student ID'].count() / total_students
percent_passing_read = percent_passing_read * 100

# Overall Passing Rate
overall_passing_rate = school_data_complete[(school_data_complete['reading_score'] >= 70) & (school_data_complete['math_score'] >= 70)].groupby('school_name')['Student ID'].count() / total_students

# School Summary 
school_summary_df = pd.DataFrame({
    "School Type": school_type,
    "Total Students": total_students,
    "Total School Budget": total_school_budget,
    "Per Student Budget": per_student_budget,
    "Average Math Score": avg_math_score,
    "Average Reading Score": avg_read_score,
    '% Passing Math': percent_passing_math,
    '% Passing Reading': percent_passing_read,
    "% Overall Passing Rate": overall_passing_rate
})

# Munging
school_summary_df = school_summary_df[['School Type',
                          'Total Students', 
                          'Total School Budget', 
                          'Per Student Budget', 
                          'Average Math Score', 
                          'Average Reading Score',
                          '% Passing Math',
                          '% Passing Reading',
                          '% Overall Passing Rate']]

# Formatting
school_summary_df['Total School Budget'] = school_summary_df['Total School Budget'].apply(lambda x: "$"+lc.format_string("%.2f",x,True))
school_summary_df['Average Math Score'] = school_summary_df['Average Math Score'].apply(lambda x: lc.format_string("%.6f",x,True))
school_summary_df['Average Reading Score'] = school_summary_df['Average Reading Score'].apply(lambda x: lc.format_string("%.5f",x,True))
school_summary_df['% Passing Math'] = school_summary_df['% Passing Math'].apply(lambda x: lc.format_string("%.6f",x,True))
school_summary_df['% Passing Reading'] = school_summary_df['% Passing Reading'].apply(lambda x: lc.format_string("%.6f",x,True))
school_summary_df['% Overall Passing Rate'] = school_summary_df['% Overall Passing Rate'].apply(lambda x: lc.format_string("%.6f",x,True))

school_summary_df

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Bailey High School,District,4976,"$3,124,928.00",628.0,77.048432,81.03396,66.680064,81.933280,0.546423
Cabrera High School,Charter,1858,"$1,081,356.00",582.0,83.061895,83.97578,94.133477,97.039828,0.913348
Figueroa High School,District,2949,"$1,884,411.00",639.0,76.711767,81.15802,65.988471,80.739234,0.532045
Ford High School,District,2739,"$1,763,916.00",644.0,77.102592,80.74626,68.309602,79.299014,0.542899
Griffin High School,Charter,1468,"$917,500.00",625.0,83.351499,83.81676,93.392371,97.138965,0.905995
Hernandez High School,District,4635,"$3,022,020.00",652.0,77.289752,80.93441,66.752967,80.862999,0.535275
Holden High School,Charter,427,"$248,087.00",581.0,83.803279,83.81499,92.505855,96.252927,0.892272
Huang High School,District,2917,"$1,910,635.00",655.0,76.629414,81.18272,65.683922,81.316421,0.535139
Johnson High School,District,4761,"$3,094,650.00",650.0,77.072464,80.96639,66.057551,81.222432,0.535392
Pena High School,Charter,962,"$585,858.00",609.0,83.839917,84.04470,94.594595,95.945946,0.905405


## Top Performing Schools (By Passing Rate)

* Sort and display the top five schools in overall passing rate

In [140]:
top_performing_schools = school_summary_df.sort_values('% Overall Passing Rate', ascending = False)

top_performing_schools = top_performing_schools.head()
top_performing_schools

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Cabrera High School,Charter,1858,"$1,081,356.00",582.0,83.061895,83.97578,94.133477,97.039828,0.913348
Thomas High School,Charter,1635,"$1,043,130.00",638.0,83.418349,83.84893,93.272171,97.308869,0.909480
Griffin High School,Charter,1468,"$917,500.00",625.0,83.351499,83.81676,93.392371,97.138965,0.905995
Wilson High School,Charter,2283,"$1,319,574.00",578.0,83.274201,83.98949,93.867718,96.539641,0.905826
Pena High School,Charter,962,"$585,858.00",609.0,83.839917,84.04470,94.594595,95.945946,0.905405


## Bottom Performing Schools (By Passing Rate)

* Sort and display the five worst-performing schools

In [129]:
bottom_performing_schools = top_performing_schools.tail()

bottom_performing_schools = bottom_performing_schools.sort_values('% Overall Passing Rate')
bottom_performing_schools

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Rodriguez High School,District,3999,"$2,547,363.00",637.0,76.842711,80.74469,66.366592,80.220055,0.529882
Figueroa High School,District,2949,"$1,884,411.00",639.0,76.711767,81.15802,65.988471,80.739234,0.532045
Huang High School,District,2917,"$1,910,635.00",655.0,76.629414,81.18272,65.683922,81.316421,0.535139
Hernandez High School,District,4635,"$3,022,020.00",652.0,77.289752,80.93441,66.752967,80.862999,0.535275
Johnson High School,District,4761,"$3,094,650.00",650.0,77.072464,80.96639,66.057551,81.222432,0.535392


## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [132]:
# Gather average math scores for each grade
ninth_grade_math_scores = student_data[student_data['grade'] == '9th'].groupby('school_name')['math_score'].mean()
tenth_grade_math_scores = student_data[student_data['grade'] == '10th'].groupby('school_name')['math_score'].mean()
eleventh_grade_math_scores = student_data[student_data['grade'] == '11th'].groupby('school_name')['math_score'].mean()
twelfth_grade_math_scores = student_data[student_data['grade'] == '12th'].groupby('school_name')['math_score'].mean()

# Math Scores Dataframe
math_scores_by_grade_df = pd.DataFrame({"9th": ninth_grade_math_scores,
                                     "10th": tenth_grade_math_scores,
                                     "11th": eleventh_grade_math_scores,
                                     "12th": twelfth_grade_math_scores})

# Munging
math_scores_by_grade_df = math_scores_by_grade_df[['9th', '10th', '11th', '12th']]

# Formatting
math_scores_by_grade_df['9th'] = math_scores_by_grade_df['9th'].apply(lambda x: lc.format_string("%.6f",x,True))
math_scores_by_grade_df['10th'] = math_scores_by_grade_df['10th'].apply(lambda x: lc.format_string("%.6f",x,True))
math_scores_by_grade_df['11th'] = math_scores_by_grade_df['11th'].apply(lambda x: lc.format_string("%.6f",x,True))
math_scores_by_grade_df['12th'] = math_scores_by_grade_df['12th'].apply(lambda x: lc.format_string("%.6f",x,True))


# Set the index
math_scores_by_grade_df.index.name = 'School'

math_scores_by_grade_df

,9th,10th,11th,12th
School,,,,
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248


## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [133]:
# Gather average reading scores for each grade 
ninth_grade_reading_scores = student_data[student_data['grade'] == '9th'].groupby('school_name')['reading_score'].mean() 
tenth_grade_reading_scores = student_data[student_data['grade'] == '10th'].groupby('school_name')['reading_score'].mean() 
eleventh_grade_reading_scores = student_data[student_data['grade'] == '11th'].groupby('school_name')['reading_score'].mean() 
twelfth_grade_reading_scores = student_data[student_data['grade'] == '12th'].groupby('school_name')['reading_score'].mean() 

# Reading Scores Dataframe 
reading_scores_by_grade_df = pd.DataFrame({"9th": ninth_grade_reading_scores, 
                                           "10th": tenth_grade_reading_scores, 
                                           "11th": eleventh_grade_reading_scores, 
                                           "12th": twelfth_grade_reading_scores}) 

# Munging 
reading_scores_by_grade_df = reading_scores_by_grade_df[['9th', '10th', '11th', '12th']] 

# Formatting 
reading_scores_by_grade_df['9th'] = reading_scores_by_grade_df['9th'].apply(lambda x: lc.format_string("%.6f",x,True)) 
reading_scores_by_grade_df['10th'] = reading_scores_by_grade_df['10th'].apply(lambda x: lc.format_string("%.6f",x,True)) 
reading_scores_by_grade_df['11th'] = reading_scores_by_grade_df['11th'].apply(lambda x: lc.format_string("%.6f",x,True)) 
reading_scores_by_grade_df['12th'] = reading_scores_by_grade_df['12th'].apply(lambda x: lc.format_string("%.6f",x,True)) 

# Set the index 
reading_scores_by_grade_df.index.name = 'School' 

reading_scores_by_grade_df 

,9th,10th,11th,12th
School,,,,
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [135]:
# Creating spending bins
bins = [0, 584.999, 614.999, 644.999, 999999]
group_name = ['< $585', "$585 - 614", "$615 - 644", "> $644"]
school_data_complete['spending_bins'] = pd.cut(school_data_complete['budget'] / school_data_complete['size'], bins, labels = group_name)

grouped_by_spending = school_data_complete.groupby('spending_bins')

# Average Math Score 
avg_math_score = grouped_by_spending['math_score'].mean()

# Average Reading Score
avg_read_score = grouped_by_spending['reading_score'].mean()

# Passing Math Score (%)
percent_passing_math = school_data_complete[school_data_complete['math_score'] >= 70].groupby('spending_bins')['Student ID'].count() / grouped_by_spending['Student ID'].count() 

# Passing Reading Score (%)
percent_passing_read = school_data_complete[school_data_complete['reading_score'] >= 70].groupby('spending_bins')['Student ID'].count() / grouped_by_spending['Student ID'].count() 

# Overall Passing Rate
overall_passing_rate = school_data_complete[(school_data_complete['reading_score'] >= 70) & (school_data_complete['math_score'] >= 70)].groupby('spending_bins')['Student ID'].count() / grouped_by_spending['Student ID'].count()

# Scores by Spending Dataframe
scores_by_spending_df = pd.DataFrame({'Average Math Score': avg_math_score,
                                      'Average Reading Score': avg_read_score,
                                      '% Passing Math': percent_passing_math,
                                      '% Passing Reading': percent_passing_read,
                                      '% Overall Passing Rate': overall_passing_rate})

# Munging 
scores_by_spending_df = scores_by_spending_df[['Average Math Score', 'Average Reading Score', '% Passing Math',
                                               '% Passing Reading', '% Overall Passing Rate']]

# Setting and naming the index
scores_by_spending_df.index.name = "Spending Ranges (Per Student)"
scores_by_spending_df = scores_by_spending_df.reindex(group_name)

# Formatting
scores_by_spending_df['Average Math Score'] = scores_by_spending_df['Average Math Score'].apply(lambda x: lc.format_string("%.6f",x,True))
scores_by_spending_df['Average Reading Score'] = scores_by_spending_df['Average Reading Score'].apply(lambda x: lc.format_string("%.5f",x,True))
scores_by_spending_df['% Passing Math'] = scores_by_spending_df['% Passing Math'].apply(lambda x: lc.format_string("%.6f",x,True))
scores_by_spending_df['% Passing Reading'] = scores_by_spending_df['% Passing Reading'].apply(lambda x: lc.format_string("%.6f",x,True))
scores_by_spending_df['% Overall Passing Rate'] = scores_by_spending_df['% Overall Passing Rate'].apply(lambda x: lc.format_string("%.6f",x,True))

scores_by_spending_df


,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Spending Ranges (Per Student),,,,,
< $585,83.363065,83.96404,0.937029,0.966866,0.906407
$585 - 614,83.529196,83.83841,0.941241,0.958869,0.901212
$615 - 644,78.061635,81.43409,0.714004,0.836148,0.602893
> $644,77.049297,81.00560,0.662308,0.811094,0.535288


## Scores by School Size

* Perform the same operations as above, based on school size.

In [137]:
# Creating spending bins
bins = [0, 999, 1999, 99999999999]
group_name = ["Small (<1000)", "Medium (1000-2000)" , "Large (>2000)"]
school_data_complete['size_bins'] = pd.cut(school_data_complete['size'], bins, labels = group_name)

grouped_by_size = school_data_complete.groupby('size_bins')

# Average Math Score 
avg_math_score = grouped_by_size['math_score'].mean()

# Average Reading Score
avg_read_score = grouped_by_size['reading_score'].mean()

# Passing Math Score (%)
percent_passing_math = school_data_complete[school_data_complete['math_score'] >= 70].groupby('size_bins')['Student ID'].count() / grouped_by_size['Student ID'].count() 

# Passing Reading Score (%)
percent_passing_read = school_data_complete[school_data_complete['reading_score'] >= 70].groupby('size_bins')['Student ID'].count() / grouped_by_size['Student ID'].count() 

# Overall Passing Rate
overall_passing_rate = school_data_complete[(school_data_complete['reading_score'] >= 70) & (school_data_complete['math_score'] >= 70)].groupby('size_bins')['Student ID'].count() / grouped_by_size['Student ID'].count()

# Scores by Spending Dataframe
scores_by_size_df = pd.DataFrame({'Average Math Score': avg_math_score,
                                      'Average Reading Score': avg_read_score,
                                      '% Passing Math': percent_passing_math,
                                      '% Passing Reading': percent_passing_read,
                                      '% Overall Passing Rate': overall_passing_rate})

# Munging 
scores_by_size_df = scores_by_size_df[['Average Math Score', 'Average Reading Score', '% Passing Math',
                                               '% Passing Reading', '% Overall Passing Rate']]

# Setting and naming the index
scores_by_size_df.index.name = "School Size"
scores_by_size_df = scores_by_size_df.reindex(group_name)

# Formatting
scores_by_size_df['Average Math Score'] = scores_by_size_df['Average Math Score'].apply(lambda x: lc.format_string("%.6f",x,True))
scores_by_size_df['Average Reading Score'] = scores_by_size_df['Average Reading Score'].apply(lambda x: lc.format_string("%.5f",x,True))
scores_by_size_df['% Passing Math'] = scores_by_size_df['% Passing Math'].apply(lambda x: lc.format_string("%.6f",x,True))
scores_by_size_df['% Passing Reading'] = scores_by_size_df['% Passing Reading'].apply(lambda x: lc.format_string("%.6f",x,True))
scores_by_size_df['% Overall Passing Rate'] = scores_by_size_df['% Overall Passing Rate'].apply(lambda x: lc.format_string("%.6f",x,True))

scores_by_size_df

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Size,,,,,
Small (<1000),83.828654,83.97408,0.939525,0.960403,0.901368
Medium (1000-2000),83.372682,83.86799,0.936165,0.967731,0.906243
Large (>2000),77.477597,81.19867,0.686524,0.821252,0.565740


## Scores by School Type

* Perform the same operations as above, based on school type.

In [139]:
grouped_by_type = school_data_complete.groupby('type')

# Average Math Score 
avg_math_score = grouped_by_type['math_score'].mean()

# Average Reading Score
avg_read_score = grouped_by_type['reading_score'].mean()

# Passing Math Score (%)
percent_passing_math = school_data_complete[school_data_complete['math_score'] >= 70].groupby('type')['Student ID'].count() / grouped_by_type['Student ID'].count() 

# Passing Reading Score (%)
percent_passing_read = school_data_complete[school_data_complete['reading_score'] >= 70].groupby('type')['Student ID'].count() / grouped_by_type['Student ID'].count() 

# Overall Passing Rate
overall_passing_rate = school_data_complete[(school_data_complete['reading_score'] >= 70) & (school_data_complete['math_score'] >= 70)].groupby('type')['Student ID'].count() / grouped_by_type['Student ID'].count()

# Scores by Spending Dataframe
scores_by_type_df = pd.DataFrame({'Average Math Score': avg_math_score,
                                  'Average Reading Score': avg_read_score,
                                  '% Passing Math': percent_passing_math,
                                  '% Passing Reading': percent_passing_read,
                                  '% Overall Passing Rate': overall_passing_rate})

# Munging 
scores_by_type_df = scores_by_type_df[['Average Math Score', 'Average Reading Score', '% Passing Math',
                                               '% Passing Reading', '% Overall Passing Rate']]

# Setting and naming the index
scores_by_type_df.index.name = "School Type"

# Formatting
scores_by_type_df['Average Math Score'] = scores_by_type_df['Average Math Score'].apply(lambda x: lc.format_string("%.6f",x,True))
scores_by_type_df['Average Reading Score'] = scores_by_type_df['Average Reading Score'].apply(lambda x: lc.format_string("%.5f",x,True))
scores_by_type_df['% Passing Math'] = scores_by_type_df['% Passing Math'].apply(lambda x: lc.format_string("%.6f",x,True))
scores_by_type_df['% Passing Reading'] = scores_by_type_df['% Passing Reading'].apply(lambda x: lc.format_string("%.6f",x,True))
scores_by_type_df['% Overall Passing Rate'] = scores_by_type_df['% Overall Passing Rate'].apply(lambda x: lc.format_string("%.6f",x,True))

scores_by_type_df

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Type,,,,,
Charter,83.406183,83.90282,0.937018,0.966459,0.905609
District,76.987026,80.96249,0.665184,0.809052,0.536959
